In [1]:
from azure.core.credentials import AzureKeyCredential
from dotenv import dotenv_values

config = dotenv_values(".env")

azure_openai_api_key = config.get("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = config.get("AZURE_OPENAI_API_BASE")
azure_openai_api_version = config.get("AZURE_OPENAI_API_VERSION")
azure_openai_chat_model = config.get("AZURE_OPENAI_MODEL")
azure_openai_embedding_model = config.get("AZURE_OPENAI_EMBEDDING_MODEL")

search_credential = AzureKeyCredential(config.get("SEARCH_KEY"))
search_endpoint = config.get("SEARCH_ENDPOINT")

index_name = config.get("SEARCH_INDEX_NAME")
index_name

'demo'

In [4]:
from azure.search.documents import SearchItemPaged

def print_results(results: SearchItemPaged[dict]):
    semantic_answers = results.get_answers()
    if semantic_answers:
        for answer in semantic_answers:
            if answer.highlights:
                print(f"Semantic Answer: {answer.highlights}")
            else:
                print(f"Semantic Answer: {answer.text}")
            print(f"Semantic Answer Score: {answer.score}\n")

    for result in results:
        print(f"Title: {result['title']}")  
        print(f"Score: {result['@search.score']}")
        if result.get('@search.reranker_score'):
            print(f"Reranker Score: {result['@search.reranker_score']}")
        print(f"Content: {result['content']}")  

        captions = result["@search.captions"]
        if captions:
            caption = captions[0]
            if caption.highlights:
                print(f"Caption: {caption.highlights}\n")
            else:
                print(f"Caption: {caption.text}\n")
        print()

In [ ]:
from azure.search.documents.models import VectorizedQuery
from openai import AzureOpenAI
from azure.search.documents import SearchClient

search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=search_credential
)

client = AzureOpenAI(
  api_key = azure_openai_api_key,  
  api_version = azure_openai_api_version,
  azure_endpoint = azure_openai_endpoint
)

# Pure Vector Search
query = "*"
embedding = client.embeddings.create(input=query, model=azure_openai_embedding_model, dimensions=3072).data[0].embedding

# 50 is an optimal value for k_nearest_neighbors when performing vector search
# To learn more about how vector ranking works, please visit https://learn.microsoft.com/azure/search/vector-search-ranking
vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=50, fields="text_vector")

results = search_client.search(
    search_text=None,
    vector_queries= [vector_query],
    select=["title", "content"],
    top=3
)


print_results(results)

Title: 1706.03762v7.pdf_chunk_6
Score: 0.5537048
Content: We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of queries and keys of dimension dk, and values of dimension dy. We compute the dot products of the query with all keys, divide each by Vdk, and apply a softmax function to obtain the weights on the :selected: values. In practice, we compute the attention function on a set of queries simultaneously, packed together into a matrix Q. The keys and values are also packed together into matrices K and V. We compute the matrix of outputs as: Attention(Q, K, V) = softmax( Vdk QKT V (1) The two most commonly used attention functions are additive attention [2], and dot-product (multi- plicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor of . Additive attention computes the compatibility function using a feed-forward network with :selected: Vdk a single hidden layer. While the two are similar in